In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np, os
import matplotlib.pyplot as plt

from pathlib import Path
# import yaml
# import re
# import datetime

# import pyarrow as pa
# import pyarrow.parquet as pq

# import gbd_mapping as gm
# from vivarium import Artifact

# from db_queries import get_ids, get_outputs, get_population, get_covariate_estimates
# from get_draws.api import get_draws

import vivarium_helpers as vh
# import vivarium_helpers.id_helper as idh
# from vivarium_helpers.vph_output.operations import VPHOperator
# from vivarium_helpers.vph_output.measures import VPHResults
from vivarium_helpers.utils import convert_to_categorical, constant_categorical, print_memory_usage, Timer
# from vivarium_helpers.vph_output import cleaning
# from vivarium_helpers.vph_output.loading import load_draws_from_keyspace_files
from vivarium_helpers.projects.alzheimers import loading, population
from vivarium_helpers.projects.alzheimers.population import RunType
# from vivarium_helpers.vph_artifact.operations import convert_to_sim_format
from vivarium_helpers.projects.alzheimers.results import AlzheimersResultsProcessor

!date
!whoami
!pwd

Fri Nov 21 01:09:22 PST 2025
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/results_tables


In [3]:
# from vivarium_helpers.projects.alzheimers.results import AlzheimersResultsProcessor

# Define data directories

In [4]:
# Project directory
project_dir = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/')

# Output directory to store final results for client
output_dir = Path(r"J:\Project\simulation_science\alzheimers\results_in_progress".replace('\\', '/').replace('J:', '/snfs1'))
print(output_dir.exists())

# Output from multistate life table
mslt_output_dir = output_dir.parent / 'results_11_14_2025_mslt'
print(mslt_output_dir.exists())
output_dir

True
True


PosixPath('/snfs1/Project/simulation_science/alzheimers/results_in_progress')

In [5]:
# # For testing: Run directory containing model 8.3 results for all
# # locations
# model_run_subdir = 'results/abie_consistent_model_test/united_states_of_america/2025_10_28_08_55_05/'

# # Results directory for model 8.3, for testing
# results_dirs = project_dir / model_run_subdir / 'results/'

# Artifact for models 8.3 - 8.7
artifact_model_number = '8.3'


In [6]:
# Model 8.4 results (final runs for 10/31/25 intermediate results)
# Each batch run contains all locations, 100 random seeds, and 3 or 4 draws
model_run_dir_8_4 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec')

# Model 8.7 results (updated final run completed on 11/10/2025)
model_run_dir_8_7 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec')

# Batch runs are stored in "timestamp" subdirectories of the model run
# directory
with os.scandir(model_run_dir_8_7) as entries:
    batch_run_dirs_8_7 = [entry.path for entry in entries]

# Select a model and filter for testing
batch_run_dirs = batch_run_dirs_8_7#[:2]

# batch_results_dirs = []
# for run_dir in batch_run_dirs:
#     if run_dir.endswith('2025_10_29_20_41_39'):
#         # One batch had to be deduplicated
#         results_dir = run_dir + '/deduplicated_results'
#     else:
#         results_dir = run_dir + '/results'
#     batch_results_dirs.append(results_dir)
# batch_results_dirs

# batch_results_dirs
batch_run_dirs

['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_06_42_37',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_07_09_05_46',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_10_07_44_18',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_06_43_53',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_13_00_54',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_16_28_26',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_07_13_14_27',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/result

# Create an AlzheimersResultsProcessor object to process results

In [7]:
results = AlzheimersResultsProcessor('8.3', RunType.FINAL, batch_run_dirs=batch_run_dirs)

In [8]:
results.initial_simulation_population

2000000

In [9]:
results.locations

['United States of America',
 'Brazil',
 'China',
 'Germany',
 'Israel',
 'Japan',
 'Spain',
 'Sweden',
 'Taiwan (Province of China)',
 'United Kingdom']

In [10]:
results.draws

[457,
 169,
 323,
 258,
 446,
 89,
 41,
 113,
 356,
 273,
 158,
 24,
 265,
 317,
 177,
 199,
 392,
 346,
 480,
 262,
 460,
 499,
 2,
 13,
 46]

# Load population data and calculate model scale

In [11]:
with Timer():
    results.load_population_data()
results.ops.describe(results.model_scale)

Elapsed time: 0:00:13.329572


,count,mean,std,min,25%,50%,75%,max
location,,,,,,,,
Brazil,25.0,1.116473,0.102910,0.886671,1.059915,1.119741,1.145108,1.386433
China,25.0,0.123020,0.007061,0.106308,0.120095,0.123499,0.125744,0.143724
Germany,25.0,0.922878,0.055810,0.854854,0.876986,0.914862,0.983316,1.012529
Israel,25.0,41.883212,2.979755,37.896818,39.739917,40.651231,43.913069,48.068295
Japan,25.0,0.586225,0.042318,0.511002,0.557443,0.585295,0.611764,0.694869
Spain,25.0,3.192170,0.169602,2.875882,3.073433,3.190730,3.289187,3.574185
Sweden,25.0,12.993787,0.944507,11.361366,12.398862,12.890094,13.665418,14.830542
Taiwan (Province of China),25.0,8.853801,0.468546,8.050507,8.487516,8.855923,9.154749,9.783891
United Kingdom,25.0,2.057251,0.093390,1.891726,1.987250,2.042450,2.134812,2.215977


In [12]:
results.person_time

,location,sex,event_year,input_draw,value,age_group
0,United States of America,Female,2025,2,1.142971e+07,25_to_29
1,United States of America,Female,2025,13,1.100320e+07,25_to_29
2,United States of America,Female,2025,24,1.143598e+07,25_to_29
3,United States of America,Female,2025,41,1.126842e+07,25_to_29
4,United States of America,Female,2025,46,1.114181e+07,25_to_29
...,...,...,...,...,...,...
569995,United Kingdom,Male,2100,446,1.254920e+05,95_plus
569996,United Kingdom,Male,2100,457,1.446562e+05,95_plus
569997,United Kingdom,Male,2100,460,1.251287e+05,95_plus
569998,United Kingdom,Male,2100,480,1.243646e+05,95_plus


# Load and process deaths data for testing

In [13]:
# with Timer():
#     results.load_measure_from_batch_runs('deaths', filters=loading.FINAL_RESULTS_FILTERS['deaths'])

In [14]:
# results.data.deaths

In [15]:
# deaths_prepped = results.process_deaths(results.data.deaths)
# deaths_prepped

# Preprocess MSLT results

In [16]:
!ls $mslt_output_dir

2025_11_14__14_11_42_outside_sim_results_draw_level.csv


In [17]:
mslt_results = pd.read_csv(mslt_output_dir / '2025_11_14__14_11_42_outside_sim_results_draw_level.csv', index_col=0)
mslt_results

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Draw,Value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,499,40424.816911


In [18]:
mslt_results.Age.unique()

array(['60_to_64', '65_to_69', '70_to_74', '75_to_79'], dtype=object)

In [71]:
mslt_results_prepped = results.process_mslt_results(mslt_results)
mslt_results_prepped

,input_draw,location,age_group,disease_stage,event_year,measure,metric,scenario,sex,value
0,2,Brazil,60_to_64,Susceptible,2025,BBBM Tests,Number,BBBM Testing Only,Female,0.000000
1,2,Brazil,65_to_69,Susceptible,2025,BBBM Tests,Number,BBBM Testing Only,Female,0.000000
2,2,Brazil,70_to_74,Susceptible,2025,BBBM Tests,Number,BBBM Testing Only,Female,0.000000
3,2,Brazil,75_to_79,Susceptible,2025,BBBM Tests,Number,BBBM Testing Only,Female,0.000000
4,2,Brazil,60_to_64,Susceptible,2025,BBBM Tests,Number,BBBM Testing Only,Male,0.000000
...,...,...,...,...,...,...,...,...,...,...
911995,499,United States of America,75_to_79,Susceptible,2100,Medication Initiation,Number,BBBM Testing and Treatment,Female,19205.694986
911996,499,United States of America,60_to_64,Susceptible,2100,Medication Initiation,Number,BBBM Testing and Treatment,Male,30474.470531
911997,499,United States of America,65_to_69,Susceptible,2100,Medication Initiation,Number,BBBM Testing and Treatment,Male,24684.321130
911998,499,United States of America,70_to_74,Susceptible,2100,Medication Initiation,Number,BBBM Testing and Treatment,Male,10523.315850


In [ ]:
comparison = results.ops.compare_values(mslt_results_prepped, mslt_results_prepped_orig)
comparison

value  \
                                                                                                                                                self   
age_group disease_stage event_year input_draw location                 measure               metric scenario                   sex                     
60_to_64  Susceptible   2030       2          Brazil                   BBBM Tests            Number BBBM Testing Only          Female  639985.107113   
65_to_69  Susceptible   2030       2          Brazil                   BBBM Tests            Number BBBM Testing Only          Female  527760.627411   
70_to_74  Susceptible   2030       2          Brazil                   BBBM Tests            Number BBBM Testing Only          Female  407019.007838   
75_to_79  Susceptible   2030       2          Brazil                   BBBM Tests            Number BBBM Testing Only          Female  283144.696852   
60_to_64  Susceptible   2030       2          Brazil                   BBBM Tests            Number BBBM Testing Only          Male    558367.079738   
...                                                                                                                                              ...   
75_to_79  Susceptible   2100       499        United States of America Medication Initiation Number BBBM Testing and Treatment Female   19205.694986   
60_to_64  Susceptible   2100       499        United States of America Medication Initiation Number BBBM Testing and Treatment Male     30474.470531   
65_to_69  Susceptible   2100       499        United States of America Medication Initiation Number BBBM Testing and Treatment Male     24684.321130   
70_to_74  Susceptible   2100       499        United States of America Medication Initiation Number BBBM Testing and Treatment Male     10523.315850   
75_to_79  Susceptible   2100       499        United States of America Medication Initiation Number BBBM Testing and Treatment Male     17994.870104   

                                                                                                                                                      
                                                                                                                                               other  
age_group disease_stage event_year input_draw location                 measure               metric scenario                   sex                    
60_to_64  Susceptible   2030       2          Brazil                   BBBM Tests            Number BBBM Testing Only          Female  559092.867600  
65_to_69  Susceptible   2030       2          Brazil                   BBBM Tests            Number BBBM Testing Only          Female  461053.232811  
70_to_74  Susceptible   2030       2          Brazil                   BBBM Tests            Number BBBM Testing Only          Female  355572.999638  
75_to_79  Susceptible   2030       2          Brazil                   BBBM Tests            Number BBBM Testing Only          Female  247356.038053  
60_to_64  Susceptible   2030       2          Brazil                   BBBM Tests            Number BBBM Testing Only          Male    487791.119378  
...                                                                                                                                              ...  
75_to_79  Susceptible   2100       499        United States of America Medication Initiation Number BBBM Testing and Treatment Female   43144.890681  
60_to_64  Susceptible   2100       499        United States of America Medication Initiation Number BBBM Testing and Treatment Male     68459.782403  
65_to_69  Susceptible   2100       499        United States of America Medication Initiation Number BBBM Testing and Treatment Male     55452.423746  
70_to_74  Susceptible   2100       499        United States of America Medication Initiation Number BBBM Testing and Treatment Male     23640.243808  
75_to_79  Susceptible   2100       499        Unite

In [ ]:
comparison[('value', 'self')] / comparison[('value', 'other')]

age_group  disease_stage  event_year  input_draw  location                  measure                metric  scenario                    sex   
60_to_64   Susceptible    2030        2           Brazil                    BBBM Tests             Number  BBBM Testing Only           Female    1.144685
65_to_69   Susceptible    2030        2           Brazil                    BBBM Tests             Number  BBBM Testing Only           Female    1.144685
70_to_74   Susceptible    2030        2           Brazil                    BBBM Tests             Number  BBBM Testing Only           Female    1.144685
75_to_79   Susceptible    2030        2           Brazil                    BBBM Tests             Number  BBBM Testing Only           Female    1.144685
60_to_64   Susceptible    2030        2           Brazil                    BBBM Tests             Number  BBBM Testing Only           Male      1.144685
                                                                                        

In [ ]:
mslt_results_prepped_orig

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,499,40424.816911


In [63]:
mslt_results_prepped.measure.unique()

['BBBM Tests', 'Positive BBBM Tests', 'Medication Initiation']
Categories (3, object): ['Positive BBBM Tests', 'BBBM Tests', 'Medication Initiation']

# BBBM Test Counts

In [20]:
filename = 'counts_bbbm_tests'
# short_name = 'bbbm_tests'
with Timer():
    bbbm_tests = results.load_measure_from_batch_runs(
        filename, False, filters=loading.FINAL_RESULTS_FILTERS[filename])

bbbm_tests

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('bbbm_test_results', '!=', 'not_tested'), ('age_group', 'in', ['60_to_64', '65_to_69', '70_to_74', '75_to_79'])]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spa

,age_group,artifact_path,bbbm_test_results,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location
0,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,baseline,Female,0.0,Japan
1,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,baseline,Male,0.0,Japan
2,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing,Female,0.0,Japan
3,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing,Male,0.0,Japan
4,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing_and_treatment,Female,0.0,Japan
...,...,...,...,...,...,...,...,...,...,...,...,...
911995,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,46,counts_bbbm_tests,baseline,Male,0.0,Brazil
911996,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,46,counts_bbbm_tests,bbbm_testing,Female,3023.0,Brazil
911997,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,46,counts_bbbm_tests,bbbm_testing,Male,1609.0,Brazil
911998,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,46,counts_bbbm_tests,bbbm_testing_and_treatment,Female,3023.0,Brazil


In [21]:
bbbm_tests.age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79']
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [22]:
bbbm_tests.query("value>0").age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79']
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [78]:
bbbm_tests_prepped = results.process_bbbm_tests(bbbm_tests, mslt_results_prepped)
bbbm_tests_prepped

,age_group,event_year,input_draw,location,measure,scenario,sex,value,disease_stage,metric
0,60_to_64,2025,2,Japan,BBBM Tests,baseline,Female,0.000000,Preclinical AD,Number
1,60_to_64,2025,2,Japan,BBBM Tests,baseline,Male,0.000000,Preclinical AD,Number
2,60_to_64,2025,2,Japan,BBBM Tests,bbbm_testing,Female,0.000000,Preclinical AD,Number
3,60_to_64,2025,2,Japan,BBBM Tests,bbbm_testing,Male,0.000000,Preclinical AD,Number
4,60_to_64,2025,2,Japan,BBBM Tests,bbbm_testing_and_treatment,Female,0.000000,Preclinical AD,Number
...,...,...,...,...,...,...,...,...,...,...
1519995,75_to_79,2100,499,United States of America,Positive BBBM Tests,BBBM Testing and Treatment,Female,64018.983285,Susceptible,Number
1519996,60_to_64,2100,499,United States of America,Positive BBBM Tests,BBBM Testing and Treatment,Male,101581.568436,Susceptible,Number
1519997,65_to_69,2100,499,United States of America,Positive BBBM Tests,BBBM Testing and Treatment,Male,82281.070433,Susceptible,Number
1519998,70_to_74,2100,499,United States of America,Positive BBBM Tests,BBBM Testing and Treatment,Male,35077.719501,Susceptible,Number


In [79]:
bbbm_tests_prepped.measure.unique()

['BBBM Tests', 'Positive BBBM Tests']
Categories (2, object): ['BBBM Tests', 'Positive BBBM Tests']

In [81]:
with Timer():
    bbbm_tests_output = results.summarize_and_beautify(bbbm_tests_prepped, 'disease_stage')
bbbm_tests_output
# 121600 rows

Timer SummarizingTimer started
2025-11-21 13:57:10.609545
2025-11-21 13:57:19.038781
2025-11-21 13:59:47.212962
Elapsed time: 156.6109 seconds
Elapsed time: 0:02:36.611287


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_64,Female,Preclinical AD,Reference,BBBM Tests,Number,0.000000,0.000000,0.000000
1,2025,Brazil,60_to_64,Male,Preclinical AD,Reference,BBBM Tests,Number,0.000000,0.000000,0.000000
2,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing Only,BBBM Tests,Number,0.000000,0.000000,0.000000
3,2025,Brazil,60_to_64,Male,Preclinical AD,BBBM Testing Only,BBBM Tests,Number,0.000000,0.000000,0.000000
4,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing and Treatment,BBBM Tests,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
121595,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Positive BBBM Tests,Number,136831.490805,127174.516102,144619.600140
121596,2100,United States of America,75_to_79,Female,Susceptible,BBBM Testing Only,Positive BBBM Tests,Rate,0.019039,0.018760,0.019285
121597,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Positive BBBM Tests,Rate,0.020939,0.020708,0.021162
121598,2100,United States of America,75_to_79,Female,Susceptible,BBBM Testing and Treatment,Positive BBBM Tests,Rate,0.019039,0.018760,0.019285


In [82]:
bbbm_tests_output.Measure.unique()

['BBBM Tests', 'Positive BBBM Tests']
Categories (2, object): ['BBBM Tests', 'Positive BBBM Tests']

In [83]:
bbbm_tests_output.Age.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79']
Categories (4, object): ['60_to_64', '65_to_69', '70_to_74', '75_to_79']

In [84]:
bbbm_tests_output.query(
    "Year == 2100 & Location == 'United States of America' & Age == '75_to_79' & Sex == 'Male' & `Disease Stage` == 'Susceptible' & Scenario == 'BBBM Testing Only' & Measure == 'Positive BBBM Tests'"
)

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
121593,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Positive BBBM Tests,Number,136831.490805,127174.516102,144619.600140
121597,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Positive BBBM Tests,Rate,0.020939,0.020708,0.021162


In [85]:
# Save BBBM testing results
bbbm_tests_output.to_csv(output_dir / 'bbbm_tests.csv', index=False)

# CSF and PET Test Counts

In [26]:
csf_pet_filters = [('testing_state', 'in', ['csf', 'pet'])]
# filename = 'counts_baseline_tests_among_eligible'
with Timer():
    csf_pet_tests = results.load_measure_from_batch_runs(
        'counts_baseline_tests_among_eligible', False, filters=csf_pet_filters
    )
print(len(csf_pet_tests), 'rows')
csf_pet_tests.head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('testing_state', 'in', ['csf', 'pet'])]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/art

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,testing_state,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Female,csf,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Female,pet,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Male,csf,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Male,pet,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,bbbm_testing,Female,csf,0.0,Japan


In [27]:
csf_pet_tests.query("value>0").age_group.unique()

['30_to_34', '35_to_39', '40_to_44', '45_to_49', '50_to_54', ..., '75_to_79', '80_to_84', '85_to_89', '90_to_94', '95_plus']
Length: 14
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [28]:
csf_pet_tests_prepped = results.process_csf_pet_tests(csf_pet_tests)
csf_pet_tests_prepped

99.186122 MB measure
33.066122 MB minuend
66.126122 MB subtrahend
21.670217 MB minuend re-indexed
43.330217 MB subtrahend re-indexed
43.330277 MB difference
75.246254 MB difference with reset index
77.526579 MB final difference


,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,testing_state,value,location,metric,disease_stage
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,CSF Tests,baseline,Female,csf,0.0,Japan,Number,MCI due to AD
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,PET Tests,baseline,Female,pet,0.0,Japan,Number,MCI due to AD
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,CSF Tests,baseline,Male,csf,0.0,Japan,Number,MCI due to AD
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,PET Tests,baseline,Male,pet,0.0,Japan,Number,MCI due to AD
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,CSF Tests,bbbm_testing,Female,csf,0.0,Japan,Number,MCI due to AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5699995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,46,Averted PET Tests,bbbm_testing,Male,pet,30.0,Brazil,Number,MCI due to AD
5699996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,46,Averted CSF Tests,bbbm_testing_and_treatment,Female,csf,58.0,Brazil,Number,MCI due to AD
5699997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,46,Averted PET Tests,bbbm_testing_and_treatment,Female,pet,72.0,Brazil,Number,MCI due to AD
5699998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,46,Averted CSF Tests,bbbm_testing_and_treatment,Male,csf,26.0,Brazil,Number,MCI due to AD


In [29]:
csf_pet_tests_output = results.summarize_and_beautify(csf_pet_tests_prepped, 'disease_stage')
csf_pet_tests_output

Timer SummarizingTimer started
2025-11-21 01:15:03.417751
2025-11-21 01:15:31.404030
2025-11-21 01:24:46.073327
Elapsed time: 582.6700 seconds


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,25_to_29,Female,MCI due to AD,Reference,CSF Tests,Number,0.000000,0.000000,0.000000
1,2025,Brazil,25_to_29,Male,MCI due to AD,Reference,CSF Tests,Number,0.000000,0.000000,0.000000
2,2025,Brazil,25_to_29,Female,MCI due to AD,BBBM Testing Only,CSF Tests,Number,0.000000,0.000000,0.000000
3,2025,Brazil,25_to_29,Male,MCI due to AD,BBBM Testing Only,CSF Tests,Number,0.000000,0.000000,0.000000
4,2025,Brazil,25_to_29,Female,MCI due to AD,BBBM Testing and Treatment,CSF Tests,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
455995,2100,United States of America,95_plus,Male,MCI due to AD,BBBM Testing and Treatment,Averted PET Tests,Number,21.627795,3.418094,45.162917
455996,2100,United States of America,95_plus,Female,MCI due to AD,BBBM Testing Only,Averted PET Tests,Rate,0.000065,0.000025,0.000112
455997,2100,United States of America,95_plus,Male,MCI due to AD,BBBM Testing Only,Averted PET Tests,Rate,0.000036,0.000006,0.000071
455998,2100,United States of America,95_plus,Female,MCI due to AD,BBBM Testing and Treatment,Averted PET Tests,Rate,0.000065,0.000025,0.000112


In [30]:
# Save CSF/PET testing results
csf_pet_tests_output.to_csv(output_dir / 'csf_pet_tests.csv', index=False)

# Treatment Counts

In [31]:
# treatments_filters = [
#     ('sub_entity', 'in',
#      ['susceptible_to_treatment_to_waiting_for_treatment',
#       'waiting_for_treatment_to_full_effect_long',
#       'waiting_for_treatment_to_full_effect_short']
#     )
# ]
treatments_filename = 'transition_count_treatment'

with Timer():
    treatments = results.load_measure_from_batch_runs(
        treatments_filename, False, 
        filters=loading.FINAL_RESULTS_FILTERS[treatments_filename]
    )
print(len(treatments), 'rows')
treatments.head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('sub_entity', 'in', ['waiting_for_treatment_to_full_effect_long', 'waiting_for_treatment_to_full_effect_short']), ('age_group', 'in', ['60_to_64', '65_to_69', '70_to_74', '75_to_79', '80_to_84'])]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simul

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,value,location
0,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,waiting_for_treatment_to_full_effect_long,0.0,Japan
1,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,waiting_for_treatment_to_full_effect_short,0.0,Japan
2,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Male,waiting_for_treatment_to_full_effect_long,0.0,Japan
3,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Male,waiting_for_treatment_to_full_effect_short,0.0,Japan
4,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,bbbm_testing,Female,waiting_for_treatment_to_full_effect_long,0.0,Japan


In [32]:
start_treatment = [
        'waiting_for_treatment_to_full_effect_long',
        'waiting_for_treatment_to_full_effect_short']
treatments.query("sub_entity in @start_treatment and value>0").age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79', '80_to_84']
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [33]:
treatments.query("sub_entity=='susceptible_to_treatment_to_waiting_for_treatment' and value>0").age_group.unique()
# Should be ['60_to_64', '65_to_69', '70_to_74', '75_to_79']

[], Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [86]:
treatments_prepped = results.process_treatments(treatments, mslt_results_prepped)
treatments_prepped

,age_group,event_year,input_draw,measure,scenario,sex,value,location,disease_stage
0,60_to_64,2025,169,Medication Completion,baseline,Female,0.000000,Japan,Preclinical AD
1,60_to_64,2025,169,Medication Discontinuation,baseline,Female,0.000000,Japan,Preclinical AD
2,60_to_64,2025,169,Medication Completion,baseline,Male,0.000000,Japan,Preclinical AD
3,60_to_64,2025,169,Medication Discontinuation,baseline,Male,0.000000,Japan,Preclinical AD
4,60_to_64,2025,169,Medication Completion,bbbm_testing,Female,0.000000,Japan,Preclinical AD
...,...,...,...,...,...,...,...,...,...
2583995,75_to_79,2100,499,Medication Initiation,BBBM Testing and Treatment,Female,19205.694986,United States of America,Susceptible
2583996,60_to_64,2100,499,Medication Initiation,BBBM Testing and Treatment,Male,30474.470531,United States of America,Susceptible
2583997,65_to_69,2100,499,Medication Initiation,BBBM Testing and Treatment,Male,24684.321130,United States of America,Susceptible
2583998,70_to_74,2100,499,Medication Initiation,BBBM Testing and Treatment,Male,10523.315850,United States of America,Susceptible


In [90]:
with Timer():
    treatments_output = results.summarize_and_beautify(treatments_prepped, 'disease_stage')

print(len(treatments_output), 'rows')
treatments_output.head()

Timer SummarizingTimer started
2025-11-21 14:04:31.953301
2025-11-21 14:04:44.349125
2025-11-21 14:07:56.651911
Elapsed time: 204.7072 seconds
Elapsed time: 0:03:24.707457
161120 rows


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_64,Female,Preclinical AD,Reference,Medication Completion,Number,0.0,0.0,0.0
1,2025,Brazil,60_to_64,Male,Preclinical AD,Reference,Medication Completion,Number,0.0,0.0,0.0
2,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing Only,Medication Completion,Number,0.0,0.0,0.0
3,2025,Brazil,60_to_64,Male,Preclinical AD,BBBM Testing Only,Medication Completion,Number,0.0,0.0,0.0
4,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing and Treatment,Medication Completion,Number,0.0,0.0,0.0


In [ ]:
treatments_output.query(
    "Year == 2100 & Location == 'United States of America' & Age == '75_to_79' & Sex == 'Male' & `Disease Stage` == 'Susceptible' & Scenario == 'BBBM Testing and Treatment' & Measure == 'Medication Initiation'"
)
# orig mean: 93092.973924
# new mean: 41049.447242

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
133755,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,41049.447242,38152.354830,43385.880042
133759,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Rate,0.006282,0.006212,0.006349


In [93]:
41049.447242 / 93092.973924

0.4409510783865631

In [92]:
# Save treatments output
treatments_output.to_csv(output_dir / 'medication.csv', index=False)

# Check some stuff

In [94]:
!ls -halt $output_dir

total 332M
-rw-rw-r-- 1 ndbs       Domain Users 19M Nov 21 14:08 medication.csv
-rw-rw-r-- 1 ndbs       Domain Users 16M Nov 21 14:00 bbbm_tests.csv
-rw-rw-r-- 1 ndbs       Domain Users 52M Nov 21 04:40 incidence.csv
drwxrwxr-x 3 ndbs       Domain Users   1 Nov 21 04:40 .
-rw-rw-r-- 1 ndbs       Domain Users 53M Nov 21 04:11 prevalence.csv
-rw-rw-r-- 1 ndbs       Domain Users 87M Nov 21 03:27 dalys.csv
-rw-rw-r-- 1 ndbs       Domain Users 30M Nov 21 03:04 deaths.csv
-rw-rw-r-- 1 ndbs       Domain Users 57M Nov 21 01:24 csf_pet_tests.csv
drwxrwxr-x 2 ndbs       Domain Users   0 Nov 17 11:53 version1
drwxrwxrwx 6 2074479877    310492440   1 Nov 14 14:37 ..


In [37]:
treatments_output

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_64,Female,Preclinical AD,Reference,Medication Completion,Number,0.000000,0.000000,0.000000
1,2025,Brazil,60_to_64,Male,Preclinical AD,Reference,Medication Completion,Number,0.000000,0.000000,0.000000
2,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing Only,Medication Completion,Number,0.000000,0.000000,0.000000
3,2025,Brazil,60_to_64,Male,Preclinical AD,BBBM Testing Only,Medication Completion,Number,0.000000,0.000000,0.000000
4,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing and Treatment,Medication Completion,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
161115,2100,United States of America,80_to_84,Male,Preclinical AD,Reference,Medication Initiation,Rate,0.000000,0.000000,0.000000
161116,2100,United States of America,80_to_84,Female,Preclinical AD,BBBM Testing Only,Medication Initiation,Rate,0.000000,0.000000,0.000000
161117,2100,United States of America,80_to_84,Male,Preclinical AD,BBBM Testing Only,Medication Initiation,Rate,0.000000,0.000000,0.000000
161118,2100,United States of America,80_to_84,Female,Preclinical AD,BBBM Testing and Treatment,Medication Initiation,Rate,0.000158,0.000017,0.000472


In [38]:
treatments_output.query("`Disease Stage` == 'Preclinical AD'").Year.unique()

[2025, 2026, 2027, 2028, 2029, ..., 2096, 2097, 2098, 2099, 2100]
Length: 76
Categories (76, int64): [2025, 2026, 2027, 2028, ..., 2097, 2098, 2099, 2100]

In [39]:
treatments_prepped.age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79', '80_to_84']
Categories (5, object): ['60_to_64', '65_to_69', '70_to_74', '75_to_79', '80_to_84']

In [40]:
treatment_rates = results.calculate_rate(treatments_prepped)
treatment_rates

,age_group,event_year,input_draw,location,sex,disease_stage,measure,scenario,value,metric
0,60_to_64,2025,2,Brazil,Female,Preclinical AD,Medication Completion,baseline,0.000000,Rate
1,60_to_64,2025,2,Brazil,Female,Preclinical AD,Medication Discontinuation,baseline,0.000000,Rate
2,60_to_64,2025,2,Brazil,Female,Preclinical AD,Medication Completion,bbbm_testing,0.000000,Rate
3,60_to_64,2025,2,Brazil,Female,Preclinical AD,Medication Discontinuation,bbbm_testing,0.000000,Rate
4,60_to_64,2025,2,Brazil,Female,Preclinical AD,Medication Completion,bbbm_testing_and_treatment,0.000000,Rate
...,...,...,...,...,...,...,...,...,...,...
2583995,80_to_84,2100,499,United States of America,Male,Preclinical AD,Medication Initiation,baseline,0.000000,Rate
2583996,80_to_84,2100,499,United States of America,Male,Preclinical AD,Medication Initiation,bbbm_testing,0.000000,Rate
2583997,80_to_84,2100,499,United States of America,Male,Preclinical AD,Medication Initiation,bbbm_testing,0.000000,Rate
2583998,80_to_84,2100,499,United States of America,Male,Preclinical AD,Medication Initiation,bbbm_testing_and_treatment,0.000073,Rate


In [41]:
treatment_rates.query("disease_stage=='Susceptible'")

,age_group,event_year,input_draw,location,sex,disease_stage,measure,scenario,value,metric
12,60_to_64,2025,2,Brazil,Female,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate
13,60_to_64,2025,2,Brazil,Female,Susceptible,Medication Initiation,BBBM Testing and Treatment,0.000000,Rate
26,60_to_64,2025,2,Brazil,Male,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate
27,60_to_64,2025,2,Brazil,Male,Susceptible,Medication Initiation,BBBM Testing and Treatment,0.000000,Rate
40,60_to_64,2025,2,China,Female,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate
...,...,...,...,...,...,...,...,...,...,...
2127971,75_to_79,2100,499,United Kingdom,Male,Susceptible,Medication Initiation,BBBM Testing and Treatment,0.012633,Rate
2127984,75_to_79,2100,499,United States of America,Female,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate
2127985,75_to_79,2100,499,United States of America,Female,Susceptible,Medication Initiation,BBBM Testing and Treatment,0.005697,Rate
2127998,75_to_79,2100,499,United States of America,Male,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate


In [42]:
mslt_results_prepped.query("measure.str.contains('Medication')")

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,499,40424.816911


In [43]:
mslt_results_prepped.measure.unique()

['BBBM Tests', 'BBBM Positive Tests', 'Medication Initiation']
Categories (3, object): ['BBBM Positive Tests', 'BBBM Tests', 'Medication Initiation']

# Scratchwork

In [44]:
bbbm_tests_prepped

,age_group,event_year,input_draw,location,measure,scenario,sex,value,disease_stage,metric
0,60_to_64,2025,2,Japan,BBBM Tests,baseline,Female,0.000000,Preclinical AD,Number
1,60_to_64,2025,2,Japan,BBBM Tests,baseline,Male,0.000000,Preclinical AD,Number
2,60_to_64,2025,2,Japan,BBBM Tests,bbbm_testing,Female,0.000000,Preclinical AD,Number
3,60_to_64,2025,2,Japan,BBBM Tests,bbbm_testing,Male,0.000000,Preclinical AD,Number
4,60_to_64,2025,2,Japan,BBBM Tests,bbbm_testing_and_treatment,Female,0.000000,Preclinical AD,Number
...,...,...,...,...,...,...,...,...,...,...
1519995,75_to_79,2100,46,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,137004.345357,Susceptible,Number
1519996,75_to_79,2100,460,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,138973.142889,Susceptible,Number
1519997,75_to_79,2100,480,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,130651.100856,Susceptible,Number
1519998,75_to_79,2100,499,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,134749.389704,Susceptible,Number


In [45]:
column_name_map = {
            'event_year': 'Year',
            'age_group': 'Age',
            'location': 'Location',
            'sex': 'Sex',
            'scenario': 'Scenario',
            'measure': 'Measure',
            'metric': 'Metric',
            'disease_stage': 'Disease Stage',
            'input_draw': 'Draw',
            'value': 'Value',
            'mean': 'Mean',
            'lower': '95% UI Lower',
            'upper': '95% UI Upper',
        }
reverse_map = {v: k for k, v in column_name_map.items()}
reverse_map

{'Year': 'event_year',
 'Age': 'age_group',
 'Location': 'location',
 'Sex': 'sex',
 'Scenario': 'scenario',
 'Measure': 'measure',
 'Metric': 'metric',
 'Disease Stage': 'disease_stage',
 'Draw': 'input_draw',
 'Value': 'value',
 'Mean': 'mean',
 '95% UI Lower': 'lower',
 '95% UI Upper': 'upper'}

In [46]:
temp = mslt_results.rename(columns=reverse_map)
temp

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,499,40424.816911


In [47]:
pd.concat([bbbm_tests_prepped, temp])

,age_group,event_year,input_draw,location,measure,scenario,sex,value,disease_stage,metric
0,60_to_64,2025,2,Japan,BBBM Tests,baseline,Female,0.000000,Preclinical AD,Number
1,60_to_64,2025,2,Japan,BBBM Tests,baseline,Male,0.000000,Preclinical AD,Number
2,60_to_64,2025,2,Japan,BBBM Tests,bbbm_testing,Female,0.000000,Preclinical AD,Number
3,60_to_64,2025,2,Japan,BBBM Tests,bbbm_testing,Male,0.000000,Preclinical AD,Number
4,60_to_64,2025,2,Japan,BBBM Tests,bbbm_testing_and_treatment,Female,0.000000,Preclinical AD,Number
...,...,...,...,...,...,...,...,...,...,...
151995,75_to_79,2100,46,United States of America,Improper Medication Uses,BBBM Testing and Treatment,Male,41101.303607,Susceptible,Number
151996,75_to_79,2100,460,United States of America,Improper Medication Uses,BBBM Testing and Treatment,Male,41691.942867,Susceptible,Number
151997,75_to_79,2100,480,United States of America,Improper Medication Uses,BBBM Testing and Treatment,Male,39195.330257,Susceptible,Number
151998,75_to_79,2100,499,United States of America,Improper Medication Uses,BBBM Testing and Treatment,Male,40424.816911,Susceptible,Number


In [48]:
temp.query("measure.str.contains('Medication') and value > 0 and scenario=='BBBM Testing Only'")

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
1000,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,113,24115.316094
1001,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,13,25432.967559
1002,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,158,24846.791963
1003,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,169,24018.184411
1004,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,177,21771.561248
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,499,40424.816911


In [49]:
temp.measure.unique()

array(['BBBM Tests', 'BBBM False Positive Tests',
       'Improper Medication Uses'], dtype=object)

In [50]:
temp.replace(
    {'measure': {'BBBM False Positive Tests': 'BBBM Positive Tests',
 'Improper Medication Uses': 'Medication Initiation'}})


,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,499,40424.816911


In [51]:
bbbm_tests_prepped.dtypes

age_group        category
event_year       category
input_draw       category
location         category
measure          category
scenario         category
sex              category
value             float64
disease_stage    category
metric           category
dtype: object

In [52]:
testing = temp.query("scenario=='BBBM Testing Only'").drop(columns='scenario')
treatment = temp.query("scenario=='BBBM Testing and Treatment'").drop(columns='scenario')
# This shows that the whole dataframe was copied to the testing-only
# scenario, including medication initiations, which is incorrect
results.ops.compare_values(testing, treatment)

age_group,disease_stage,event_year,input_draw,location,measure,metric,sex


In [53]:
mslt_results_processed = results.process_mslt_results(mslt_results)
mslt_results_processed

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,499,40424.816911


In [54]:
mslt_results_processed.input_draw.unique()

[113, 13, 158, 169, 177, ..., 46, 460, 480, 499, 89]
Length: 25
Categories (25, int64): [2, 13, 24, 41, ..., 457, 460, 480, 499]

In [55]:
print_memory_usage(mslt_results_processed)

22.806047 MB 


In [56]:
mslt_results_processed.measure.unique()

['BBBM Tests', 'BBBM Positive Tests', 'Medication Initiation']
Categories (3, object): ['BBBM Positive Tests', 'BBBM Tests', 'Medication Initiation']

In [57]:
!date

Fri Nov 21 01:29:32 PST 2025
